# Sine wave. Forward kinematics. 

In the previous practice we dealt with step signal. Let's dive into our workout routine with array of elements as a reference. For the sake of simplicity, within this script we deal with sine wave. 

As always we start with uploading needed libraries

In [1]:
from os import path
import mujoco
import mujoco_viewer
import numpy as np

Let us declare constant variables such as simulation time `SIMEND`, step numbers `STEP_NUM`, name of  xml file `MODEL_NAME`, and MuJoCo entities: `model` and `data`

In [2]:
SIMEND = 10
TIMESTEP = 0.002
STEP_NUM = int(SIMEND/TIMESTEP)
MODEL_NAME = 'two_rot_robot.xml'

ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, '')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

Our first activity will involve implementing position-velocity control for the servo.

To understand the next piece of code you need to open the xml file and to take a look at `actuator` container. You'll see there a few actuators: (0) `motor`, (1) `position`, (2) velocity for the joint A, and (3) `motor`, (4) `position`, (5) `velocity` for the joint B. To implement servo control we need only position and velocity types, i.e., actuators with IDs 1 and 2 for the joint A, and with IDs 4 and 5 for the joint B. Ids 0 and 3 are needed for torque control.

In [3]:
def set_position_servo(mjmodel, actuator_no, kp):
    mjmodel.actuator_gainprm[actuator_no, 0] = kp
    mjmodel.actuator_biasprm[actuator_no, 1] = -kp


def set_velocity_servo(mjmodel, actuator_no, kv):
    mjmodel.actuator_gainprm[actuator_no, 0] = kv
    mjmodel.actuator_biasprm[actuator_no, 2] = -kv

def controller(mjmodel, mjdata, ref, i):

    ID_POS_A = 1
    ID_VEL_A = 2
    ID_POS_B = 4
    ID_VEL_B = 5
    ID_JOINT_A = 0
    ID_JOINT_B = 1

    set_position_servo(mjmodel, ID_POS_A, 90)
    set_velocity_servo(mjmodel, ID_VEL_A, 2)
    mjdata.ctrl[ID_POS_A] = ref[ID_JOINT_A][i]
    
    set_position_servo(mjmodel, ID_POS_B, 90)
    set_velocity_servo(mjmodel, ID_VEL_B, 2)
    mjdata.ctrl[ID_POS_B] = ref[ID_JOINT_B][i]

At this stage we have to create a viewer entity, set control callback, and write a piece of code for the `mj_step` command

In [4]:
timeseries = np.linspace(0, SIMEND, STEP_NUM)

AMP_1 = np.pi/6 
AMP_2 = - np.pi/3
FREQ = 1
BIAS_1 = np.pi/4
BIAS_2 = - np.pi/2

ref = [AMP_1 * np.sin(FREQ * timeseries) + BIAS_1,
       AMP_2 * np.sin(FREQ * timeseries) + BIAS_2]

viewer = mujoco_viewer.MujocoViewer(model, data,
                                    title = "2R_robot",
                                    width = 1920,
                                    height = 1080)

for i in range(STEP_NUM):
    if viewer.is_alive:
        mujoco.set_mjcb_control(controller(model, data, ref, i))
        mujoco.mj_step(model, data)
        viewer.render()
    else:
        break

viewer.close()

For extra practice you can make some plots to compare reference and actual motion.